In [1]:
from fish import (
    FishLocation, FPoint, FVector,
    lon_normalize, next_location,
    HarvestPattern, InBoundPattern,
    OutBoundPattern, TrawlingPattern
)

import pickle
import random
import pandas as pd
import hashlib
import datetime

In [2]:
# datetime.datetime(2015, 2, 4, 14, 41, 59)
reference_datetime = 1423032119
datetimespan = 4 * 24 * 60 * 60

# Simulated Ports

In [3]:
# ports = dict(
#     chilung=FishLocation('port', 'CHI-LUNG', FPoint(121.75, 25.7), FVector(0.03, 0.05)),
# #     losangels=FishLocation('port', 'LOS-ANGELES', FPoint(-118.18, 33.75), FVector(0.03, 0.05)),
#     korsakov=FishLocation('port', 'KORSAKOV', FPoint(142.75, 46.39), FVector(0.6, 0.6)),
#     abashiri=FishLocation('port', 'ABASHIRI', FPoint(144.28, 44.02), FVector(0.5, 0.3)),
# #     acapulco=FishLocation('port', 'ACAPULCO', FPoint(-99.93, 16.85), FVector(0.07, 0.05)),
# )
# pickle.dump(ports, open('ports.pkl', 'wb'))

# Simulated Fisheries

In [4]:
# lat_bound = FVector(20, 40)
# lon_bound = FVector(140, 170)
# size_bound = FVector(3, 6)
# n = 30
# fisheries = dict()
# for i in range(1, n+1):
# #     x = random.uniform(lon_bound.x, lon_bound.y)
#     name = 'fishery{:02d}'.format(i)
#     fisheries[name] = FishLocation(
#         'fishery', name.upper(),
#         FPoint(random.uniform(lon_bound.x, lon_bound.y), random.uniform(lat_bound.x, lat_bound.y)),
#         FVector(random.uniform(size_bound.x, size_bound.y), random.uniform(size_bound.x, size_bound.y))
#     )

# pickle.dump(fisheries, open('fisheries.pkl', 'wb'))

In [3]:
fisheries = pickle.load(open('fisheries.pkl', 'rb'))
ports = pickle.load(open('ports.pkl', 'rb'))

In [6]:
# fishery_data = [(k, v.type, v.name, v.center.x, v.center.y, v.radius.x) for k, v in fisheries.items()]
# fishery_csv = pd.DataFrame(fishery_data, columns=['uuid', 'type', 'tag', 'longitude', 'latitude', 'radius'])
# fishery_csv.to_csv('fishery.csv', index=False)

In [7]:
# port_data = [(k, v.type, v.name, v.center.x, v.center.y, v.radius.x) for k, v in ports.items()]
# port_csv = pd.DataFrame(port_data, columns=['uuid', 'type', 'tag', 'longitude', 'latitude', 'radius'])
# port_csv.to_csv('port.csv', index=False)

In [4]:
port_keys = list(ports.keys())
fishery_keys = list(fisheries.keys())

In [5]:
num_vessels = 50


In [7]:
total_rows = []

for k in range(num_vessels):
    mmsi = 'mmsi{:d}'.format(k + 1)  # hashlib.md5(str(datetime.datetime.now()).encode('utf-8')).hexdigest()
    trawling_probability = random.random() * 0.1
    initial = ports[random.choice(port_keys)]
    finish = ports[random.choice(port_keys)]
    activities = FVector(5, 9)
    num_activity = round(random.uniform(activities.x, activities.y))
    
    locations = []
    for i in range(num_activity):
        position = locations[-1].sampling() if len(locations) > 0 else initial.sampling()
        locations.append(next_location(fishery_keys, fisheries, position=position, nearest=True))
    
    when = reference_datetime + int(round(random.uniform(0, datetimespan)))
    sequences = []
    sequences.extend(OutBoundPattern(initial, when, initial=initial.sampling()).generate(FVector(5.5, 0.5)))
    for loc in locations:
        ini = FPoint(sequences[-1].longitude, sequences[-1].latitude)
        sofar = sequences[-1].timestemp
        if random.random() > trawling_probability:
            sequences.extend(HarvestPattern(loc, sofar, initial=ini).generate())
        else:
            sequences.extend(TrawlingPattern(loc, sofar, initial=ini).generate())
    ini = FPoint(sequences[-1].longitude, sequences[-1].latitude)
    sofar = sequences[-1].timestemp
    sequences.extend(InBoundPattern(finish, sofar, initial=ini).generate())
    
    data = [(seq.label, seq.timestemp, seq.longitude, seq.latitude) for seq in sequences]
    
    df = pd.DataFrame(data, columns=['tag', 'timestamp', 'longitude', 'latitude'])
    df['uuid'] = mmsi
    df['type'] = 'vessel'
    
    total_rows.append(df)
    
    print('# {}-th vessel tack generated.'.format(k + 1))


# 1-th vessel tack generated.
# 2-th vessel tack generated.
# 3-th vessel tack generated.
# 4-th vessel tack generated.
# 5-th vessel tack generated.
# 6-th vessel tack generated.
# 7-th vessel tack generated.
# 8-th vessel tack generated.
# 9-th vessel tack generated.
# 10-th vessel tack generated.
# 11-th vessel tack generated.
# 12-th vessel tack generated.
# 13-th vessel tack generated.
# 14-th vessel tack generated.
# 15-th vessel tack generated.
# 16-th vessel tack generated.
# 17-th vessel tack generated.
# 18-th vessel tack generated.
# 19-th vessel tack generated.
# 20-th vessel tack generated.
# 21-th vessel tack generated.
# 22-th vessel tack generated.
# 23-th vessel tack generated.
# 24-th vessel tack generated.
# 25-th vessel tack generated.
# 26-th vessel tack generated.
# 27-th vessel tack generated.
# 28-th vessel tack generated.
# 29-th vessel tack generated.
# 30-th vessel tack generated.
# 31-th vessel tack generated.
# 32-th vessel tack generated.
# 33-th vessel ta

In [8]:
total_rows = pd.concat(total_rows)

In [9]:
total_rows.shape

(2417273, 6)

In [10]:
total_rows.head()

,tag,timestamp,longitude,latitude,uuid,type
0,outbound,1423291063,144.744650,44.028953,mmsi1,vessel
1,outbound,1423291205,144.533270,43.732336,mmsi1,vessel
2,outbound,1423291311,143.884993,44.103033,mmsi1,vessel
3,outbound,1423291767,143.957997,43.818445,mmsi1,vessel
4,outbound,1423291901,144.119650,44.089645,mmsi1,vessel


In [21]:
d = datetime.datetime.fromtimestamp(1423043120)
print(d.isoformat())
d.strftime('%Y-%m-%d-%H')

2015-02-04T17:45:20


'2015-02-04-17'

In [23]:
datestr = total_rows.timestamp.map(lambda row: datetime.datetime.fromtimestamp(row).strftime('%Y-%m-%d-%H'))

In [24]:
total_rows['datestr'] = datestr

In [11]:
total_rows.to_csv('vessels_50.csv', index=False)